In [2]:
import pandas as pd
import os.path 
import csv
from interface import input_path, input_load_date
from datetime import datetime, timedelta    
from routes_export import export_new_routes_files
pd.options.mode.chained_assignment = None
from interface import * 
from fix_routes_converter  import *
from flex_routes_converter import *
from tl_routes_converter import *
from HnN_routes_converter import *
from routes_export import *
from time import sleep


In [3]:
PATH = r"C:\Users\ROKOTYEV\Desktop\Тест 2\Копия Формат - Фиксированные маршруты от HN_.xlsx"
DATE_OF_LOAD = '28.11.2023'

df_rtm  = HnN_import_rtm_file(PATH)        
df_rtm = HnN_rename_columns(df_rtm)  
df_rtm = HnN_filtering_rtm(df_rtm)     
df_rtm = check_wrong_order_type(df_rtm) 
df_rtm = check_wrong_route_type(df_rtm)       
df_rtm = HnN_unpivot_rtm(df_rtm)
df_calender = HnN_create_calender(DATE_OF_LOAD)
df_routes_HnN_fix = HnN_converting_FIX_rtm_to_routes(df_rtm, df_calender)
df_routes_HnN_flex = HnN_converting_FLEX_rtm_to_routes(df_rtm)
df_routes = HnN_union_FLEX_n_FIX_routes(df_routes_HnN_fix, df_routes_HnN_flex)
df_routes = add_empty_shipto_HnN(df_routes, DATE_OF_LOAD)
df_routes = route_type_substitution(df_routes)
df_routes = errors_correction(df_routes)

export_new_routes_files(df_routes, PATH, 'Маршруты HnN','', 'Test')      





Данные сохранены в файл: C:\Users\ROKOTYEV\Desktop\Тест 2\Маршруты HnN\Test\DMT_User273_SetClientServiceMatrix_Batch.txt


In [ ]:
df_rtm  = HnN_import_rtm_file(PATH)        
df_rtm = HnN_rename_columns(df_rtm) 
df_rtm = HnN_filtering_rtm(df_rtm)  
df_rtm

In [ ]:
df_routes_in_case = df_routes.groupby(by=['AGENCY_NAME', 'ROUTE_NAME', 'EXT_ROUTE_ID','ROUTE_TYPE']).agg('size').reset_index().drop(columns=0)
df_routes_in_case['FIRST_VISIT_DATE'] = pd.to_datetime(DATE_OF_LOAD, format='%d.%m.%Y')
df_routes_in_case['FIRST_VISIT_DATE'] = df_routes_in_case['FIRST_VISIT_DATE'].astype(str)
df_routes_in_case['REPEAT_DAYS']    = '7'
df_routes_in_case['VISIT_NUMBER']   = '1'
df_routes_in_case['VISIT_DURATION'] = '130' 
df_routes_in_case['SHIP_TO'] = ''
df_routes_in_case[['WEEKDAY_NUMBER', 'DAY_NAME', 'WEEK_NUMBER', 'WEEK_1234_ORDER', 'PHOTOS_TARGET', 'DOCUMENTS_TARGET', 'PHOTOAUDIT_TARGET', 'END_DATE']] = ''
df_routes_in_case = df_routes_in_case[['SHIP_TO', 'ROUTE_NAME', 'FIRST_VISIT_DATE', 'WEEKDAY_NUMBER', 'DAY_NAME', 'WEEK_NUMBER', 'WEEK_1234_ORDER', 'EXT_ROUTE_ID',  'VISIT_NUMBER', 'AGENCY_NAME', 'VISIT_DURATION', 'ROUTE_TYPE',  'PHOTOS_TARGET', 'DOCUMENTS_TARGET', 'PHOTOAUDIT_TARGET', 'END_DATE',  'REPEAT_DAYS']]

df_routes['FIRST_VISIT_DATE'] = df_routes['FIRST_VISIT_DATE'].astype(str)
df_routes[['SHIP_TO','EXT_ROUTE_ID', 'VISIT_NUMBER', 'VISIT_DURATION', 'PHOTOS_TARGET', 'DOCUMENTS_TARGET', 'PHOTOAUDIT_TARGET', 'REPEAT_DAYS']] = df_routes[['SHIP_TO','EXT_ROUTE_ID', 'VISIT_NUMBER', 'VISIT_DURATION', 'PHOTOS_TARGET', 'DOCUMENTS_TARGET', 'PHOTOAUDIT_TARGET', 'REPEAT_DAYS']].astype('int').astype('str')
df_routes_HnN = pd.concat([df_routes_in_case, df_routes])
df_routes_HnN.reset_index(inplace=True, drop=True)


#df_routes_HnN['FIRST_VISIT_DATE'] = df_routes_HnN['FIRST_VISIT_DATE'].dt.strftime('%Y-%m-%d')
#df_routes_HnN['FIRST_VISIT_DATE'] = df_routes_HnN['FIRST_VISIT_DATE'].astype('str')
#df_routes_HnN['END_DATE'] = pd.to_datetime(df_routes_HnN['END_DATE'])
#df_routes_HnN['END_DATE'] = df_routes_HnN['END_DATE'].dt.strftime('%Y-%m-%d')
#df_routes_HnN['END_DATE'] = df_routes_HnN['END_DATE'].astype('str')


#df_routes_HnN#['SHIP_TO'].unique()
df_routes_HnN


In [ ]:
df_routes_in_case['FIRST_VISIT_DATE'] = df_routes_in_case['FIRST_VISIT_DATE'].astype(str)


In [ ]:
df_routes.dtypes

In [ ]:
import pandas as pd
import os.path 
import csv
from interface import input_path, input_load_date
from datetime import datetime, timedelta    
from routes_export import export_new_routes_files
pd.options.mode.chained_assignment = None


def HnN_import_rtm_file(PATH):
    df_rtm = pd.read_excel(PATH, sheet_name='MergeDATA', usecols='A:AM')#, engine = 'pyxlsb')
    df_rtm = df_rtm.replace(r'\n','', regex=True)
    return df_rtm

PATH = r"C:\Users\ROKOTYEV\Desktop\TEST\Формат - Фиксированные маршруты от Danone.xlsx"
df_rtm = HnN_import_rtm_file(PATH)

In [ ]:
def HnN_rename_columns(df_rtm):
    renames = {'Тип маршрута'                  :  'ROUTE_TYPE', 
               'ТТ SAP Код'                    :  'SHIP_TO', 
               'Наименование маршрута'         :  'ROUTE_NAME', 
               'Ответственный'                 :  'ROUTE_ID', 
               	'Длительность визита (минут)'  :  'VISIT_DURATION',
                'Агенство'                     :  'AGENCY_NAME',
               'Weekly Frequency'              :  'FREQUENCY_RTM',
               'РОЛЬ'                          :  'ROLE',
               'Форма заказа'                  :  'ORDER_TYPE',
               'СЦЕНАРИЙ'                      :  'SCENARIO'}
    df_rtm.rename(columns=renames, inplace=True)

    columns_list = ['ROUTE_TYPE','SHIP_TO','ROUTE_NAME','ROUTE_ID','FREQUENCY_RTM','AGENCY_NAME','VISIT_DURATION','ROLE','ORDER_TYPE','SCENARIO',
                    'ПН1', 'ВТ1', 'СР1', 'ЧТ1', 'ПТ1', 'СБ1', 'ВС1', 
                    'ПН2', 'ВТ2', 'СР2', 'ЧТ2', 'ПТ2', 'СБ2', 'ВС2',
                    'ПН3', 'ВТ3', 'СР3', 'ЧТ3', 'ПТ3', 'СБ3', 'ВС3', 
                    'ПН4', 'ВТ4', 'СР4', 'ЧТ4', 'ПТ4', 'СБ4', 'ВС4' ]
    df_rtm = df_rtm[columns_list]
    df_rtm = df_rtm.query("ROUTE_TYPE in ('TSS', 'ASM', 'SV') & ORDER_TYPE in ('FLEX', 'FIX')")
    df_rtm['SHIP_TO'] = df_rtm['SHIP_TO'].astype('float').astype('int').astype('str')
    df_rtm['VISIT_DURATION'] = df_rtm['VISIT_DURATION'].astype('float').astype('int') 
    return df_rtm


df_rtm = HnN_rename_columns(df_rtm)
df_rtm.head(2)

In [ ]:
df_rtm.loc[df_rtm['ROUTE_TYPE'] == 'TSS', 'ROUTE_TYPE'] = 'мерчендайзер'
df_rtm.loc[df_rtm['ROUTE_TYPE'] == 'ASM', 'ROUTE_TYPE'] = 'Тим Лид'

In [ ]:
df_rtm[['ROUTE_TYPE', 'ROLE']]

In [ ]:
def check_wrong_order_type(df_rtm):
    df_check_wrong_order_type = df_rtm.groupby(by=['ROUTE_NAME', 'ORDER_TYPE']).agg('size').reset_index().drop(columns=0)
    df_check_wrong_order_type = df_check_wrong_order_type.groupby(by=['ROUTE_NAME']).agg('size').reset_index()
    df_check_wrong_order_type = df_check_wrong_order_type.loc[df_check_wrong_order_type[0] > 1]

    print("ВНИМАНИЕ! Эти маршруты не будут сконвертированы, так как сожержат не верный формат заказа: FIX и FLEX одновеменно")
    for _ in df_check_wrong_order_type['ROUTE_NAME'].to_list():
        print(_)

    df_rtm = pd.merge(df_rtm, df_check_wrong_order_type['ROUTE_NAME'], how='outer', on='ROUTE_NAME', indicator=True)
    df_rtm = df_rtm.query("_merge != 'both'").drop(columns='_merge')
    return df_rtm


In [ ]:
def check_wrong_route_type(df_rtm):
    df_check_wrong_order_type = df_rtm.groupby(by=['ROUTE_NAME', 'ROUTE_TYPE']).agg('size').reset_index().drop(columns=0)
    df_check_wrong_order_type = df_check_wrong_order_type.groupby(by=['ROUTE_NAME']).agg('size').reset_index()
    df_check_wrong_order_type = df_check_wrong_order_type.loc[df_check_wrong_order_type[0] > 1]

    print("ВНИМАНИЕ! Эти маршруты не будут сконвертированы, так как содержат задвоение типа маршрута: например, ASM и TSS одновеменно")
    for _ in df_check_wrong_order_type['ROUTE_NAME'].to_list():
        print(_)

    df_rtm = pd.merge(df_rtm, df_check_wrong_order_type['ROUTE_NAME'], how='outer', on='ROUTE_NAME', indicator=True)
    df_rtm = df_rtm.query("_merge != 'both'").drop(columns='_merge')
    return df_rtm

In [ ]:
def HnN_unpivot_rtm(df_rtm):
    df_rtm_tss = pd.melt(df_rtm, id_vars=['ROUTE_TYPE','SHIP_TO','ROUTE_NAME','ROUTE_ID','FREQUENCY_RTM','AGENCY_NAME','VISIT_DURATION','ROLE','ORDER_TYPE','SCENARIO'])
    renames = {'variable'  :  'DAY_NAME', 
               'value'     :  'VISIT_NUMBER'} 
    df_rtm_tss.rename(columns=renames, inplace=True)
    filt = df_rtm_tss['VISIT_NUMBER'].notnull()
    df_rtm_tss = df_rtm_tss[filt]
    df_rtm_tss['WEEK_1234_ORDER'] = df_rtm_tss['DAY_NAME'].str[-1]
    df_rtm_tss['WEEK_1234_ORDER'] = df_rtm_tss['WEEK_1234_ORDER'].astype('int')
    df_rtm_tss['DAY_NAME']        = df_rtm_tss['DAY_NAME'].str[0:2]
    df_rtm_tss['DAY_NAME']        = df_rtm_tss['DAY_NAME'].astype('str')
    df_rtm_tss['EXT_ROUTE_ID']    = df_rtm_tss['ROUTE_NAME'].str.split('_').str[-1].str.strip()
    df_rtm_tss['ROW_NUMBER']      = df_rtm_tss.groupby(by=['ROUTE_NAME','DAY_NAME','WEEK_1234_ORDER' ]).cumcount()+1
    df_rtm_tss.loc[df_rtm_tss['VISIT_NUMBER'] == 1, ['VISIT_NUMBER']] = df_rtm_tss['ROW_NUMBER']
    df_rtm_tss['SHIP_TO'] = df_rtm_tss['SHIP_TO'].astype('float').astype('int').astype('str')
    return df_rtm_tss

df_rtm = HnN_unpivot_rtm(df_rtm)
df_rtm.head(2)

In [ ]:
#Формируем календарь на год вперед
def HnN_create_calender(DATE_OF_LOAD):
    df_calender = pd.DataFrame(pd.date_range('2023-11-27', '2024-12-31'), columns=['DATE'] )
    df_calender['WEEKDAY_NUMBER'] = df_calender['DATE'].dt.weekday+1
    df_calender['DAY_NAME'] = df_calender['WEEKDAY_NUMBER'] 
    df_calender['WEEK_NUMBER'] = df_calender['DATE'].dt.strftime("%W").astype('int')+ 1  #еще можно так .dt.isocalendar().week, но тут не стандартные номера недель
    df_calender['WEEK_1234_ORDER'] = ((df_calender['WEEK_NUMBER']  / 4)  - (df_calender['WEEK_NUMBER']  // 4)) * 4
    df_calender.loc[df_calender['WEEK_1234_ORDER'] == 0 ,   'WEEK_1234_ORDER'] = 4
    df_calender['WEEK_1234_ORDER'] = df_calender['WEEK_1234_ORDER'].astype('int')
    df_calender.replace({'DAY_NAME' : { 1 : 'ПН', 2 : 'ВТ', 3 : 'СР', 4 : 'ЧТ', 5 : 'ПТ', 6 : 'СБ', 7 : 'ВС'}}, inplace=True)
    DATE_OF_END = pd.to_datetime(DATE_OF_LOAD, format='%d.%m.%Y') +  + timedelta(days=60)  #Пробрасываем маршруты на этот перид
    filt = (df_calender['DATE'] >= pd.to_datetime(DATE_OF_LOAD, dayfirst=True)) & (df_calender['DATE'] <= DATE_OF_END)
    df_calender = df_calender[filt]
    return df_calender

DATE_OF_LOAD = "28.11.2023"
df_calender = HnN_create_calender(DATE_OF_LOAD)
df_calender.head(2)

In [ ]:
def HnN_converting_FIX_rtm_to_routes(df_rtm, df_calender): 
   #Оставляем только часть заказа с фиксированными маршрутами
    df_rtm_keys = df_rtm.query("ROUTE_TYPE in ('TSS') & ORDER_TYPE in ('FIX')")
   #Создаем матрицу: всем маршруто-точки и все дни
    df_rtm_keys = df_rtm.groupby(by=['SHIP_TO', 'ROUTE_NAME']).agg('size').reset_index().drop(columns=0)
    df_rtm_keys = df_rtm_keys[['SHIP_TO', 'ROUTE_NAME']]
    df_routes = pd.merge(df_rtm_keys, df_calender, how='cross' )
   #Добавляем к матрице заказ и удаляем лишние (пустые) строки
    df_routes = pd.merge(df_routes, df_rtm[['SHIP_TO', 'ROUTE_NAME', 'EXT_ROUTE_ID', 'DAY_NAME', 'WEEK_1234_ORDER', 'VISIT_NUMBER', 'AGENCY_NAME', 'VISIT_DURATION', 'ROUTE_TYPE']], how='left',  on=['SHIP_TO', 'ROUTE_NAME', 'DAY_NAME', 'WEEK_1234_ORDER'])
    df_routes.dropna(subset=['VISIT_NUMBER'], inplace=True)
   #Дозаполняем необходимые поля
    df_routes.rename(columns={'DATE': 'FIRST_VISIT_DATE'}, inplace=True)                                                                                                                                                    
    df_routes.loc[ df_routes['VISIT_DURATION'] < 5, ['VISIT_DURATION']]= '5'
    df_routes['VISIT_DURATION'] = df_routes['VISIT_DURATION'].astype('float').astype('int').astype('str')   
    df_routes['VISIT_NUMBER'] = df_routes['VISIT_NUMBER'].astype('int').astype('str') 
    df_routes['EXT_ROUTE_ID'] = '0000' + df_routes['EXT_ROUTE_ID'].astype('str')
    df_routes['EXT_ROUTE_ID'] = df_routes['EXT_ROUTE_ID'].str[-4:]  
    df_routes['PHOTOS_TARGET'] = '10'     # - Задай им 1 или 0, Система будет их требовать, нужна какая-нибудь циферка, если хотим совсем невалидное можно -1 указать, допустимо любое число.
    df_routes['DOCUMENTS_TARGET'] = '8'   # - Задай им 1 или 0, Система будет их требовать, нужна какая-нибудь циферка, если хотим совсем невалидное можно -1 указать, допустимо любое число.
    df_routes['PHOTOAUDIT_TARGET']= '1'   # - Задай им 1 или 0, Система будет их требовать, нужна какая-нибудь циферка, если хотим совсем невалидное можно -1 указать, допустимо любое число.
    df_routes['END_DATE']  = '2024-12-31'
    df_routes['REPEAT_DAYS']  = '0'  # Все  визиты  грузим как разовые, без повторений
    df_routes['SHIP_TO'] = df_routes['SHIP_TO'].astype('float').astype('int').astype('str')
    df_routes[' FIRST_VISIT_DATE'] = df_routes['FIRST_VISIT_DATE'].astype('str')
   #Упорядочиваем данные
    df_routes.drop(columns=['WEEKDAY_NUMBER', 'DAY_NAME', 'WEEK_NUMBER', 'WEEK_1234_ORDER'], inplace=True)
    df_routes = df_routes[['SHIP_TO', 'ROUTE_NAME', 'FIRST_VISIT_DATE', 'EXT_ROUTE_ID','VISIT_NUMBER', 'AGENCY_NAME', 'VISIT_DURATION', 'ROUTE_TYPE','PHOTOS_TARGET', 'DOCUMENTS_TARGET', 'PHOTOAUDIT_TARGET', 'END_DATE','REPEAT_DAYS']]
    return df_routes

df_routes_tss_fix = HnN_converting_FIX_rtm_to_routes(df_rtm, df_calender)
df_routes_tss_fix

In [ ]:
Замена АСМов на Тим Лид"... и ТСС на "Мерч"
df_rtm.query("ROUTE_TYPE in ('TSS', 'ASM', 'SV') & ORDER_TYPE in ('FLEX', 'FIX')")

In [ ]:
def HnN_converting_FLEX_rtm_to_routes(df_rtm):
    df_rtm.query("ORDER_TYPE in ('FLEX')")     #ROUTE_TYPE in ('TSS') &
    df_rtm = df_rtm.groupby(by=['AGENCY_NAME', 'ROUTE_NAME', 'ROUTE_TYPE', 'SHIP_TO']).agg('size').reset_index().drop(columns=0)
    df_rtm['EXT_ROUTE_ID'] = df_rtm['ROUTE_NAME'].str.split('_').str[-1].str.strip()
    df_rtm['EXT_ROUTE_ID'] = '0000' + df_rtm['EXT_ROUTE_ID'].astype('str')
    df_rtm['EXT_ROUTE_ID'] = df_rtm['EXT_ROUTE_ID'].str[-4:]
    df_rtm['FIRST_VISIT_DATE'] = '2024-12-30'
    df_rtm['REPEAT_DAYS']    = '0'
    df_rtm['VISIT_NUMBER']   = '1'
    df_rtm['VISIT_DURATION'] = '100' 
    df_rtm['PHOTOS_TARGET'] = '5' 
    df_rtm['DOCUMENTS_TARGET'] = '5' 
    df_rtm['PHOTOAUDIT_TARGET'] = '1' 
    df_rtm['END_DATE'] = '2024-12-31'
    df_routes = df_rtm[['SHIP_TO', 'ROUTE_NAME', 'FIRST_VISIT_DATE', 'EXT_ROUTE_ID','VISIT_NUMBER', 'AGENCY_NAME', 'VISIT_DURATION', 'ROUTE_TYPE','PHOTOS_TARGET', 'DOCUMENTS_TARGET', 'PHOTOAUDIT_TARGET', 'END_DATE','REPEAT_DAYS']]
    return df_routes

df_routes_tss_flex = HnN_converting_FLEX_rtm_to_routes(df_rtm)
df_routes_tss_flex

In [ ]:
def HnN_union_FLEX_n_FIX_routes(df_routes_tss_fix, df_routes_tss_flex):
    df_routes = pd.concat([df_routes_tss_fix, df_routes_tss_flex])    
    return df_routes

df_routes = HnN_union_FLEX_n_FIX_routes(df_routes_tss_fix, df_routes_tss_flex)

In [ ]:
se_routes_tss_fix = pd.Series(df_routes_tss_fix.columns, name='Col')
se_routes_tss_flex = pd.Series(df_routes_tss_flex.columns, name='Col')
pd.merge(se_routes_tss_fix, se_routes_tss_flex, how='outer', indicator=True)

In [ ]:
DATE_OF_LOAD = "28.11.2023"
def add_empty_shipto(df_routes, DATE_OF_LOAD):
    #Добавляем пустую точку чтобы обозначить системе, что данны маршрут надо целиком перезаписать
    #Danone|TestTerritory_3_Dub33|Dub33|Мерчандайзер||2023-05-08|7|1|130||||
    df_routes_in_case = df_routes.groupby(by=['AGENCY_NAME', 'ROUTE_NAME', 'EXT_ROUTE_ID','ROUTE_TYPE']).agg('size').reset_index().drop(columns=0)
    df_routes_in_case['FIRST_VISIT_DATE'] = pd.to_datetime(DATE_OF_LOAD, format='%d.%m.%Y')
    df_routes_in_case['REPEAT_DAYS']    = '7'
    df_routes_in_case['VISIT_NUMBER']   = '1'
    df_routes_in_case['VISIT_DURATION'] = '130' 
    df_routes_in_case['SHIP_TO'] = ''
    df_routes_in_case[['WEEKDAY_NUMBER', 'DAY_NAME', 'WEEK_NUMBER', 'WEEK_1234_ORDER', 'PHOTOS_TARGET', 'DOCUMENTS_TARGET', 'PHOTOAUDIT_TARGET', 'END_DATE']] = ''
    df_routes_in_case = df_routes_in_case[['SHIP_TO', 'ROUTE_NAME', 'FIRST_VISIT_DATE', 'WEEKDAY_NUMBER', 'DAY_NAME', 'WEEK_NUMBER', 'WEEK_1234_ORDER', 'EXT_ROUTE_ID',  'VISIT_NUMBER', 'AGENCY_NAME', 'VISIT_DURATION', 'ROUTE_TYPE',  'PHOTOS_TARGET', 'DOCUMENTS_TARGET', 'PHOTOAUDIT_TARGET', 'END_DATE',  'REPEAT_DAYS']]
    df_routes_flex = pd.concat([df_routes_in_case, df_routes])
    df_routes['FIRST_VISIT_DATE'] = pd.to_datetime(df_routes['FIRST_VISIT_DATE'])
    df_routes['FIRST_VISIT_DATE'] = df_routes['FIRST_VISIT_DATE'].dt.strftime('%Y-%m-%d')
    df_routes['FIRST_VISIT_DATE'] = df_routes['FIRST_VISIT_DATE'].astype('str')
    df_routes['END_DATE'] = pd.to_datetime(df_routes['END_DATE'])
    df_routes['END_DATE'] = df_routes['END_DATE'].dt.strftime('%Y-%m-%d')
    df_routes['END_DATE'] = df_routes['END_DATE'].astype('str')
    df_routes[['SHIP_TO','EXT_ROUTE_ID', 'VISIT_NUMBER', 'VISIT_DURATION', 'PHOTOS_TARGET', 'DOCUMENTS_TARGET', 'PHOTOAUDIT_TARGET', 'REPEAT_DAYS']] = df_routes[['SHIP_TO','EXT_ROUTE_ID', 'VISIT_NUMBER', 'VISIT_DURATION', 'PHOTOS_TARGET', 'DOCUMENTS_TARGET', 'PHOTOAUDIT_TARGET', 'REPEAT_DAYS']].astype('int').astype('str')
    return df_routes

df_routes = add_empty_shipto(df_routes, DATE_OF_LOAD)

df_routes

In [ ]:
export_HnN_routes_files